<a href="https://colab.research.google.com/github/VSeema/All_projects_of_Data_Science/blob/main/Names_entity_Recognition_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kamalkraj/BERT-NER.git

Cloning into 'BERT-NER'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 249 (delta 0), reused 0 (delta 0), pack-reused 246
Receiving objects: 100% (249/249), 1.67 MiB | 9.57 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [2]:
ls -l

total 12
drwxr-xr-x 8 root root 4096 Sep 19 06:00 BERT-NER/
drwx------ 5 root root 4096 Sep 19 05:59 drive/
drwxr-xr-x 1 root root 4096 Sep 16 13:40 sample_data/


In [3]:
cd BERT-NER/

/content/BERT-NER


In [4]:
ls -l

total 104
-rw-r--r-- 1 root root   470 Sep 19 06:00 api.py
-rw-r--r-- 1 root root  4674 Sep 19 06:00 bert.py
drwxr-xr-x 3 root root  4096 Sep 19 06:00 cpp-app/
drwxr-xr-x 2 root root  4096 Sep 19 06:00 data/
drwxr-xr-x 2 root root  4096 Sep 19 06:00 img/
-rw-r--r-- 1 root root 34523 Sep 19 06:00 LICENSE.txt
drwxr-xr-x 2 root root  4096 Sep 19 06:00 out_ner/
-rw-r--r-- 1 root root  4777 Sep 19 06:00 README.md
-rw-r--r-- 1 root root   173 Sep 19 06:00 requirements.txt
-rw-r--r-- 1 root root 27023 Sep 19 06:00 run_ner.py


In [5]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 176 kB 5.2 MB/s 
     |████████████████████████████████| 748.9 MB 678 bytes/s 
     |████████████████████████████████| 48 kB 5.2 MB/s 
     |████████████████████████████████| 1.5 MB 56.0 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 1.2 MB 51.0 MB/s 
     |████████████████████████████████| 131 kB 53.2 MB/s 
     |████████████████████████████████| 895 kB 46.0 MB/s 
     |████████████████████████████████| 7.9 MB 33.6 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 138 kB 53.2 MB/s 
     |████████████████████████████████| 127 kB 53.8 MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449922 sha256=647d823c26c61452252764b413943177ffbbad3fdb5e2d853914cdd80cf5513c
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built nltk
  Attempting uninstall: urll

In [6]:
!python run_ner.py --data_dir=data/ --bert_model=bert-base-cased --task_name=ner --output_dir=out_ner --max_seq_length=128 --do_train --num_train_epochs 3 --do_eval --warmup_proportion=0.1

09/19/2021 06:02:54 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
09/19/2021 06:02:55 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpas65tw4d
100% 213450/213450 [00:00<00:00, 1871067.91B/s]
09/19/2021 06:02:55 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpas65tw4d to cache at /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
09/19/2021 06:02:55 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
09/19/2021 06:02:55 - INFO - pytorch_transformers.file_utils -   removing t

In [7]:
pwd

'/content/BERT-NER'

In [8]:
%%writefile bert.py
"""BERT NER Inference."""

from __future__ import absolute_import, division, print_function

import json
import os

import torch
import torch.nn.functional as F
from nltk import word_tokenize
from pytorch_transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)

class BertNer(BertForTokenClassification):

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, valid_ids=None):
        sequence_output = self.bert(input_ids, token_type_ids, attention_mask, head_mask=None)[0]
        batch_size,max_len,feat_dim = sequence_output.shape
        valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32,device='cuda' if torch.cuda.is_available() else 'cpu')
        for i in range(batch_size):
            jj = -1
            for j in range(max_len):
                    if valid_ids[i][j].item() == 1:
                        jj += 1
                        valid_output[i][jj] = sequence_output[i][j]
        sequence_output = self.dropout(valid_output)
        logits = self.classifier(sequence_output)
        return logits

class Ner:

    def __init__(self,model_dir: str):
        self.model , self.tokenizer, self.model_config = self.load_model(model_dir)
        self.label_map = self.model_config["label_map"]
        self.max_seq_length = self.model_config["max_seq_length"]
        self.label_map = {int(k):v for k,v in self.label_map.items()}
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device)
        self.model.eval()

    def load_model(self, model_dir: str, model_config: str = "model_config.json"):
        model_config = os.path.join(model_dir,model_config)
        model_config = json.load(open(model_config))
        model = BertNer.from_pretrained(model_dir)
        tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=model_config["do_lower"])
        return model, tokenizer, model_config

    def tokenize(self, text: str):
        """ tokenize input"""
        words = word_tokenize(text)
        tokens = []
        valid_positions = []
        for i,word in enumerate(words):
            token = self.tokenizer.tokenize(word)
            tokens.extend(token)
            for i in range(len(token)):
                if i == 0:
                    valid_positions.append(1)
                else:
                    valid_positions.append(0)
        return tokens, valid_positions

    def preprocess(self, text: str):
        """ preprocess """
        tokens, valid_positions = self.tokenize(text)
        ## insert "[CLS]"
        tokens.insert(0,"[CLS]")
        valid_positions.insert(0,1)
        ## insert "[SEP]"
        tokens.append("[SEP]")
        valid_positions.append(1)
        segment_ids = []
        for i in range(len(tokens)):
            segment_ids.append(0)
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        while len(input_ids) < self.max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            valid_positions.append(0)
        return input_ids,input_mask,segment_ids,valid_positions

    def predict(self, text: str):
        input_ids,input_mask,segment_ids,valid_ids = self.preprocess(text)
        input_ids = torch.tensor([input_ids],dtype=torch.long,device=self.device)
        input_mask = torch.tensor([input_mask],dtype=torch.long,device=self.device)
        segment_ids = torch.tensor([segment_ids],dtype=torch.long,device=self.device)
        valid_ids = torch.tensor([valid_ids],dtype=torch.long,device=self.device)
        with torch.no_grad():
            logits = self.model(input_ids, segment_ids, input_mask,valid_ids)
        logits = F.softmax(logits,dim=2)
        logits_label = torch.argmax(logits,dim=2)
        logits_label = logits_label.detach().cpu().numpy().tolist()[0]

        logits_confidence = [values[label].item() for values,label in zip(logits[0],logits_label)]

        logits = []
        pos = 0
        for index,mask in enumerate(valid_ids[0]):
            if index == 0:
                continue
            if mask == 1:
                logits.append((logits_label[index-pos],logits_confidence[index-pos]))
            else:
                pos += 1
        logits.pop()

        labels = [(self.label_map[label],confidence) for label,confidence in logits]
        words = word_tokenize(text)
        assert len(labels) == len(words)

        Person = []
        Location = []
        Organization = []
        Miscelleneous = []

        for word, (label, confidence) in zip(words, labels):
            if label=="B-PER" or label=="I-PER":
                Person.append(word)
            elif label=="B-LOC" or label=="I-LOC":
                Location.append(word)
            elif label=="B-ORG" or label=="I-ORG":
                Organization.append(word)
            elif label=="B-MISC" or label=="I-MISC":
                Miscelleneous.append(word)
            else:
                output = None

        output = []
        for word, (label, confidence) in zip(words, labels):      
            if label == "B-PER":
                output.append(' '.join(Person) + ": Person")
            if label=="B-LOC":
                output.append(' '.join(Location) + ": Location")
            if label=="B-MISC":
                output.append(' '.join(Miscelleneous) + ": Miscelleneous Entity")
            if label=="B-ORG":
                output.append(' '.join(Organization) + ": Organization")
                
        return output



Overwriting bert.py


In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
!pip install pytorch-transformers

In [18]:
from bert import Ner
model = Ner("out_ner/")

#text ="Conflict of interest Name is an employee and stockholder of Novo Nordisk A/S."
#text ="I have conflicting interests with Microsoft, Google and Myntra"
#text = "My name is Seema. And Viswanatha is my father's name"

text=["Conflict of interest Name is an employee and stockholder of Novo Nordisk A/S.","I have conflicting interests with Microsoft, Google and Myntra",
      "My name is Seema. And Viswanatha is my father's name"]

for i in text:
  print("Text to predict Entity ----->", i)
  output = model.predict(i)
  for prediction in output:
    print(prediction)

Text to predict Entity -----> Conflict of interest Name is an employee and stockholder of Novo Nordisk A/S.
Novo Nordisk A/S: Organization
Text to predict Entity -----> I have conflicting interests with Microsoft, Google and Myntra
Microsoft Google Myntra: Organization
Microsoft Google Myntra: Organization
Microsoft Google Myntra: Organization
Text to predict Entity -----> My name is Seema. And Viswanatha is my father's name
Seema Viswanatha: Person
Seema Viswanatha: Person
